In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np 
import pandas as pd 
import plotly as py
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
import colorlover as cl
import plotly.graph_objs as go
import os
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from IPython.display import HTML
import calendar
import datetime
from datetime import datetime, timedelta
init_notebook_mode(connected=True)
print(os.listdir("../input"))
reviews="../input/googleplaystore_user_reviews.csv"
store="../input/googleplaystore.csv"
r = pd.read_csv(reviews)
s = pd.read_csv(store)
s.dtypes
s.head()


**Data Cleaning **

Before analysing or visualising the data, we will need to check every feature and clean the data. 


In [ ]:
 '''Cleaning data''' 
'''Reviews''' 
s['Reviews'].value_counts()
#Finding values which have letter (M)
s[s['Reviews'].str.contains('M')]
#Getting red of (M)
s.drop(s.index[10472],inplace=True)
#Converting dtype to Int
s['Reviews']=s['Reviews'].astype(int)
s.dtypes
s['Reviews'].value_counts()




In [ ]:
 '''Cleaning data''' 
'''Size'''
s['Size'].value_counts()
#Getting red of (M)
s['Size'] = s['Size'].map(lambda x: x.rstrip('M'))
#we have almost 1695 records without any value 
#we will replace these records with the mean of the size feature.
#save all sizes in new value
numsize=s[s[['Size']].apply(lambda x: x[0].isdigit(), axis=1)]
#create dataframe
newsize=numsize.Size
#convert it to float
newsize=newsize.astype(float)
#get the mean 
newsize.mean()

In [ ]:
import scipy as sp
#replacing "Varies with device" with mean of sizes
sp.stats.trim_mean(newsize,.1)
#adding the mean as the size for "varies with device"
s.loc[s['Size'].str.contains('Varies with device'), 'Size'] = '34M'

#Finding how many  Kilobyte's K are existing in size 
s.Size.str.contains("k").value_counts()
#replacing K's with .5
s.loc[s['Size'].str.contains('k'), 'Size'] = '.5M'
#remove remaining (M)s
s['Size'] = s['Size'].map(lambda x: x.rstrip('M'))
s['Size'].value_counts() 

In [ ]:
 '''Cleaning data''' 
'''Current Ver'''

s['Current Ver'].value_counts()
s['Current Ver'].isna().sum()
# replace na values with 4.1 (since majority of apps have version (4.1))
s['Current Ver']=s['Current Ver'].fillna(value=4.1)



In [ ]:
 '''Cleaning data''' 
'''Installs'''
s['Installs'].value_counts()
#Geting red of (+) signs  
s['Installs'] = s['Installs'].map(lambda x: x.rstrip('+'))
#Geting red of (,) commas  
s['Installs']  = s['Installs'] .str.replace(',', '')
#Coverting values to int datatype
s['Installs'] =s['Installs'] .astype(float)
s['Installs'].value_counts()

In [ ]:
 '''Cleaning data''' 
'''Price'''
#Getting rid of ($) signs
s['Price'] = s['Price'].replace({'\$':''}, regex = True)
s['Price'].value_counts()



In [ ]:
 '''Cleaning data''' 
'''removing all na values '''
s = s.dropna()

#removing one record which has inaccurate rate and category, and nan content rating 
s['Category'].value_counts()
s[s.Category == 1.9]
s = s.drop(s.index[10472])
s.iloc[10471:10475]
s['Category'].value_counts()


**Data Visualization**
Using seaborn's boxplot to visualize the correlation between the  rating  and the category of applications. 


**Average Ratings by Category**

In [ ]:
s['Content Rating'].value_counts()

In [ ]:
catlist = list(s['Category'].unique())
ratings = []

for cat in catlist:
    x = s[s['Category'] == cat]
    rating_rate = x.Rating.sum()/len(x)
    ratings.append(rating_rate)
data = pd.DataFrame({'Category':catlist, 'Rating':ratings})
new_index = (data['Rating'].sort_values(ascending=False)).index.values
sorted_data = data.reindex(new_index)

sorted_data

**Plotting results **

In [ ]:
plt.figure(figsize=(25,15))
sns.barplot(x=sorted_data.Category, y=sorted_data.Rating)

plt.xticks(rotation = 45)
plt.xlabel('Application Category')
plt.ylabel('Ratings')
plt.title('Average Ratings by Category')
plt.show()

**Visualization - Distribution by Sentiment **

In [ ]:
alpha_color=0.5
pos = (r[r.Sentiment == 'Positive'].count().iloc[0] /r['Sentiment'].count() * 100).round()
neg = (r[r.Sentiment  == 'Negative'].count().iloc[0] / r['Sentiment'].count() * 100).round()
neu = (r[r.Sentiment == 'Neutral'].count().iloc[0] /r['Sentiment'].count() * 100).round()

colors = cl.scales['3']['qual']['Paired']

labels = ['Positive','Negative','Neutral']
values = [pos, neg, neu]

trace = go.Pie(
    labels=labels,
    values=values,
    hoverinfo='label+percent', textinfo='percent', 
    textfont=dict(size=20, color='white'),
    marker=dict(colors=cl.to_rgb( colors ))
)

data = [trace]
layout = go.Layout(
    title='<b>Distribution by Sentiment <b>',
     margin=go.layout.Margin(
        l=173,
        r=80,
        t=100,
        b=80,
        pad=4
    )
)

fig = go.Figure(data=data, layout=layout)
py.offline.iplot(fig)


**Visualization - Distribution by App Type **

In [ ]:
free = (s[s.Type == 'Free'].count().iloc[0] / s['App'].count() * 100).round()
paid = (s[s.Type == 'Paid'].count().iloc[0] / s['App'].count() * 100).round()
labels = ['Free','Paid']
values = [free, paid]

trace = go.Pie(
    labels=labels, 
    values=values,
    hoverinfo='label+percent', textinfo='percent', 
    textfont=dict(size=20, color='white')
)

data = [trace]

layout = go.Layout(
    title='<b>Distribution by App Type <b>',
     margin=go.layout.Margin(
        l=173,
        r=80,
        t=100,
        b=80,
        pad=4
    )
)

fig = go.Figure(data=data, layout=layout)
py.offline.iplot(fig)

**Visualization - Installs by Categories **

In [ ]:
cat_install = pd.DataFrame(s, columns=['Category','Installs'])
cat_install = cat_install.groupby('Category').sum().sort_values(by='Installs', ascending=False)
cat_install.head()


In [ ]:


colors_scale = cl.scales['10']['div']['RdYlGn']
colors = cl.interp( colors_scale, 40 ) 

trace = go.Bar(
    x = cat_install.index,
    y = cat_install['Installs'],
    name = 'Installs by Categories',
    marker={'color': cl.to_rgb( colors )}
   
)

data = [trace]
layout = go.Layout(
    title='<b>Installs by Categories<b>',
     margin=go.layout.Margin(
        l=173,
        r=80,
        t=100,
        b=200,
        pad=4
    ),
    xaxis=dict(
        title='<b>Category<b>',
        tickangle=-45,
        titlefont=dict(
            size=14,
            color='rgb(107,107,107)'
        ),
    ),
    yaxis=dict(
        title='<b>Installs<b>',
        titlefont=dict(
            size=14,
            color='rgb(107,107,107)'
        ),
    )
)

fig = go.Figure(data=data, layout=layout)
py.offline.iplot(fig)



** 5 Rated Apps by Genres **

In [ ]:
(s[s['Rating']==5]['Genres'].value_counts().sort_index()/len(s)).plot(kind='bar')

plt.xlabel('Genres')
plt.ylabel('')
plt.title('5 Rated Apps by Genres')
plt.show()

**Apps by Ratings **

In [ ]:
bins=[0,1,2,3,4,5]

s['RatingB']=pd.cut(s['Rating'],bins)
s[s['Type']=='Free']['RatingB'].value_counts().sort_index().plot(kind='bar')
plt.xlabel('Ratings')
plt.ylabel('Number of Apps')
plt.title('Apps by Ratings')
plt.show()

In [ ]:
free = s[s.Type == "Free"]
paid = s[s.Type == "Paid"]
paid_head = paid.head()
paid_head

In [ ]:
corr = s.corr()
corr

In [ ]:
sns.heatmap(corr, annot=True, fmt=".2f",
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)

In [ ]:
subset_df = s[s.Category.isin(['GAME', 'FAMILY', 'PHOTOGRAPHY', 'MEDICAL', 'TOOLS', 'FINANCE',
                                 'LIFESTYLE','BUSINESS'])]
sns.set_style('darkgrid')
fig, ax = plt.subplots()
fig.set_size_inches(10, 8)
p = sns.stripplot(x="Price", y="Category", data=subset_df, jitter=True, linewidth=1)
title = ax.set_title('App pricing trend across categories')